In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), df['cardio'], random_state=41)
feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[-1.12257477, -1.5034125 ,  0.81804974, ...,  0.        ,
         0.        ,  0.        ],
       [-1.07506066,  0.56366794,  0.6100762 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.29739608,  0.07729607,  0.74872523, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.37026176, -0.7738547 , -0.84573862, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.50532108, -0.4090758 , -0.15249347, ...,  0.        ,
         0.        ,  1.        ],
       [-0.46549928,  0.19888904, -0.15249347, ...,  0.        ,
         0.        ,  1.        ]])

#### Логистическая регрессия (LogisticRegression)

In [5]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state=41)),
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7813846794997775+-0.008137519654598194


In [6]:
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
ix = np.argmax(fscore)
precision_log, recall_log, fscore_log = precision[ix], recall[ix], fscore[ix]

roc_auc_score_log = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
roc_auc_score_log

0.7882648193658791

#### Случайный лес (RandomForestClassifier)

In [7]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state=41)),
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7733864647187316+-0.00820357924439192


In [8]:
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
ix = np.argmax(fscore)
precision_rf, recall_rf, fscore_rf = precision[ix], recall[ix], fscore[ix]

roc_auc_score_rf = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
roc_auc_score_rf

0.7726476116194899

#### Сравним полученные модели

In [9]:
summary = pd.DataFrame(data = {'Metrics' : ['Precision', 'Recall', 'F-score', 'Roc_auc'],
                               'LogisticRegression' : [precision_log, recall_log, fscore_log, roc_auc_score_log],
                               'RandomForestClassifier': [precision_rf, recall_rf, fscore_rf, roc_auc_score_rf]},
                               dtype = np.float).T
summary

,0,1,2,3
Metrics,Precision,Recall,F-score,Roc_auc
LogisticRegression,0.660509,0.831453,0.736188,0.788265
RandomForestClassifier,0.644366,0.821667,0.722295,0.772648


В случае с представленными данными логистическая регрессия работает намного быстрее и показывает лучшие результаты по всем метрикам.